<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Регулярные-выражения" data-toc-modified-id="Регулярные-выражения-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Регулярные выражения</a></span></li><li><span><a href="#Лематизация" data-toc-modified-id="Лематизация-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Лематизация</a></span></li><li><span><a href="#Делим-на-выборки" data-toc-modified-id="Делим-на-выборки-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Делим на выборки</a></span></li><li><span><a href="#Токенизация" data-toc-modified-id="Токенизация-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Токенизация</a></span></li><li><span><a href="#Трубопровод" data-toc-modified-id="Трубопровод-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Трубопровод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Catboost" data-toc-modified-id="Catboost-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Catboost</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Проверка-на-адекватность" data-toc-modified-id="Проверка-на-адекватность-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Проверка на адекватность</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Задача:
Определение токсичности комментариев

## Подготовка

In [ ]:
# Импортируем необходимые библиотеки
import catboost as cb
import nltk
import numpy as np
import pandas as pd
import re
import spacy
import swifter

from google.colab import drive
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
#Загружаем данные
drive.mount('/content/gdrive')
try:
    data = pd.read_csv(r'D:\DataHome\toxic_comments.csv', index_col=0)
except:
    data = pd.read_csv('/content/toxic_comments.csv', index_col=0)
data.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161852 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    161852 non-null  object
 1   toxic   161852 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.7+ MB


В нашем распоряжении 160 тысяч комментариев, пропусков нет

### Регулярные выражения

In [ ]:
# Функция для отчистки текста от лишних символов
def clear_text(text):
    return " ".join(re.sub(r'[^a-zA-Z\'.!?#",]', ' ', text).split())

In [ ]:
data['text'] = data['text'].apply(clear_text)

In [ ]:
data

,text,toxic
0,Explanation Why the edits made under my userna...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,""" More I can't make any real suggestions on im...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159446,""" And for the second time of asking, when your...",0
159447,You should be ashamed of yourself That is a ho...,0
159448,"Spitzer Umm, theres no actual article for pros...",0
159449,And it looks like it was actually you who put ...,0


In [ ]:
data['text'] = [x.lower() for x in data['text']]

In [ ]:
data

,text,toxic
0,explanation why the edits made under my userna...,0
1,d'aww! he matches this background colour i'm s...,0
2,"hey man, i'm really not trying to edit war. it...",0
3,""" more i can't make any real suggestions on im...",0
4,"you, sir, are my hero. any chance you remember...",0
...,...,...
159446,""" and for the second time of asking, when your...",0
159447,you should be ashamed of yourself that is a ho...,0
159448,"spitzer umm, theres no actual article for pros...",0
159449,and it looks like it was actually you who put ...,0


### Лематизация

In [ ]:
# Загрузка предварительно обученной модели для английского языка
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

In [ ]:
# Функция для лемматизации текста
def lemmatize_text(text):
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return ' '.join(lemmas)

In [ ]:
data['lemmatized_text'] = data['text'].swifter.apply(lemmatize_text)

/usr/local/lib/python3.10/dist-packages/swifter/swifter.py:87: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  warnings.warn(


Pandas Apply:   0%|          | 0/161852 [00:00<?, ?it/s]

In [ ]:
data

,text,toxic,lemmatized_text
0,explanation why the edits made under my userna...,0,explanation why the edit make under my usernam...
1,d'aww! he matches this background colour i'm s...,0,d'aww ! he match this background colour I be s...
2,"hey man, i'm really not trying to edit war. it...",0,"hey man , I be really not try to edit war . it..."
3,""" more i can't make any real suggestions on im...",0,""" more I can not make any real suggestion on i..."
4,"you, sir, are my hero. any chance you remember...",0,"you , sir , be my hero . any chance you rememb..."
...,...,...,...
159446,""" and for the second time of asking, when your...",0,""" and for the second time of asking , when you..."
159447,you should be ashamed of yourself that is a ho...,0,you should be ashamed of yourself that be a ho...
159448,"spitzer umm, theres no actual article for pros...",0,"spitzer umm , there s no actual article for pr..."
159449,and it looks like it was actually you who put ...,0,and it look like it be actually you who put on...


### Делим на выборки

In [ ]:
features = data.drop(['toxic', 'text'], axis=1)
target = data['toxic']

In [ ]:
train_features, test_features = train_test_split(features, test_size=0.2, random_state=12345)
train_target, test_target = train_test_split(target, test_size=0.2, random_state=12345)

In [ ]:
train_features = train_features['lemmatized_text'].to_list()
test_features = test_features['lemmatized_text'].to_list()

### Токенизация

In [ ]:
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Делаем мешок слов
# stop_words = list(stopwords.words('english'))
# count_tf_idf = TfidfVectorizer(stop_words=stop_words)
# bow_train = count_tf_idf.fit_transform(train_features)
# bow_test = count_tf_idf.transform(test_features)

In [ ]:
#bow_train.shape, bow_test.shape

Мы загрузили все необходимые библиотеки и открыли данные, удалили из текстов лишние символы, которые могут плохо повлиять на качество предсказания, лемматизировали тексты, поделили на обучающие и тестовые выборки, токенизировали и векторизировали данные

### Трубопровод

In [ ]:
log_reg_pipe = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words=stop_words)),
    ('model', LogisticRegression())
])

In [ ]:
cb_classifier_pipe = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words=stop_words)),
    ('model', cb.CatBoostClassifier(eval_metric='F1',early_stopping_rounds = 20, verbose = 10))
])

## Обучение

У нас задача классификации, так как целевой признак может быть 1 или 0. Воспользуемся двумя моделями: логистической регрессией и catboost

In [ ]:
best_model = None
best_score = 0
best_params = None
best_estimator = None

### Логистическая регрессия

In [ ]:
param_grid = {
    'model__C': [1, 10, 100],
    'model__max_iter' : [400, 800, 1400]
}

grid = GridSearchCV(estimator=log_reg_pipe, param_grid=param_grid, cv=3, scoring='f1', error_score='raise')
grid.fit(train_features, train_target)

#Присваивает только лучшую модель
if grid.best_score_ > best_score:
    best_score = grid.best_score_
    best_params = grid.best_params_
    best_estimator = grid.best_estimator_

print("Лучшие параметры:", grid.best_params_)
print("Лучший результат:", grid.best_score_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Лучшие параметры: {'model__C': 10, 'model__max_iter': 400}
Лучший результат: 0.7770319531299276


### Catboost

In [ ]:
param_grid = {
    'model__n_estimators': [300, 700],
    'model__depth': range(6, 8, 1)
    }

grid = GridSearchCV(estimator=cb_classifier_pipe, param_grid=param_grid, cv=3, scoring = 'f1')

grid.fit(train_features, train_target)

#Присваивает только лучшую модель
if grid.best_score_ > best_score:
    best_score = grid.best_score_
    best_params = grid.best_params_
    best_estimator = grid.best_estimator_

print("Лучшие параметры:", grid.best_estimator_)
print("Лучший результат:", grid.best_score_)

Learning rate set to 0.208513
0:	learn: 0.4434852	total: 2.71s	remaining: 13m 32s
10:	learn: 0.5688383	total: 19.3s	remaining: 8m 28s
20:	learn: 0.6120870	total: 35.9s	remaining: 7m 57s
30:	learn: 0.6418990	total: 52.4s	remaining: 7m 34s
40:	learn: 0.6648979	total: 1m 9s	remaining: 7m 19s
50:	learn: 0.6763546	total: 1m 26s	remaining: 7m 4s
60:	learn: 0.6915269	total: 1m 43s	remaining: 6m 43s
70:	learn: 0.7011429	total: 1m 59s	remaining: 6m 24s
80:	learn: 0.7083865	total: 2m 15s	remaining: 6m 5s
90:	learn: 0.7162515	total: 2m 31s	remaining: 5m 47s
100:	learn: 0.7247128	total: 2m 47s	remaining: 5m 29s
110:	learn: 0.7311469	total: 3m 4s	remaining: 5m 14s
120:	learn: 0.7362439	total: 3m 21s	remaining: 4m 57s
130:	learn: 0.7439268	total: 3m 37s	remaining: 4m 40s
140:	learn: 0.7491263	total: 3m 53s	remaining: 4m 23s
150:	learn: 0.7522911	total: 4m 9s	remaining: 4m 6s
160:	learn: 0.7575221	total: 4m 25s	remaining: 3m 49s
170:	learn: 0.7600869	total: 4m 41s	remaining: 3m 32s
180:	learn: 0.7621

### Тестирование

In [ ]:
#Лучшая модель по точности
best_estimator

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('model', LogisticRegression(C=10, max_iter=400))])

In [ ]:

#Лучшие параметры самой точной модели
best_params

{'model__C': 10, 'model__max_iter': 400}

In [ ]:
#Результат точности лучшей модели на обучающей выборке
"Лучший результат", best_score

('Лучший результат', 0.7770319531299276)

In [ ]:
preds = best_estimator.predict(test_features)
"Точность", f1_score(test_target, preds)

('Точность', 0.7908146161582299)

### Проверка на адекватность

In [ ]:
dummyreg = DummyClassifier(strategy = 'uniform', random_state=12345)
dummyreg.fit(train_features, train_target)
preds = dummyreg.predict(test_features)
scores = f1_score(test_target, preds)
scores

0.16809394390031282

Лучшие результаты нам удалось получить с помощью логистической регрессии, f1 на тестовой выборке равен ~0.79. Модель прошла тест на адеватность, так как f1 больше 0.17

## Выводы

Мы очистили тексты от лишних символов, лематизировали их, поделили на выборке, токенизировали и векторизировали. Для определения токсичности текста мы воспользовались двумя моделями: логистической регрессией и бустингом catboost. Лучший результат получился при обучении логистической регрессии, при которой f1 оказался равен 0.79